# Setup

In [1]:
#!pip install --pre pycaret
#!pip install  boruta

In [2]:
# libs


from boruta import BorutaPy
from pycaret.datasets import get_data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV, f_classif, chi2
import warnings
warnings.filterwarnings('ignore')

In [3]:
# dados
df_credit = get_data('credit')


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,90000,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
2,50000,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
3,50000,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0


## Boruta

Para ilustrar o funcionamento dos métodos, vamos criar um dataset. Isso mesmo! Datasets sintéticos são legais para demonstrar esse tipo de coisa. Na prática, não conhecemos a verdadeira relação funcional entre as variáveis que estamos modelando, certo? Tampouco sabemos se as features que ali estão ajudam ou não o nosso modelo. A ideia de criar um dataset sintético é justamente ter a certeza de quais features têm relação com o target e quais não têm!. Dessa formaa, podemos checar a validade desse método.

vamos criar uma lista de variáveis.

In [4]:
variaveis_dataset = [
   'linear',            # 1
   'quadrado',          # 2
   'seno',              # 3
   'interacao1',        # 4
   'interacao2',        # 5
   'interacao3',        # 6
   'aleatoria_1',       # 7
   'aleatoria_2',       # 8
   'aleatoria_3',       # 9
   'aleatoria_4']       # 10

Cada variavel será uma distribuição normal e o dataset é composto por 20000 linhas.

In [5]:
np.random.seed(0)
X = pd.DataFrame(np.random.normal(size = (20000 , len(variaveis_dataset))) , columns =  variaveis_dataset)

Y será uma função de algumas colunas de X (todas, menos as aleatorias)

In [6]:
y = X['linear'] + X['quadrado']**2 + np.sin(3*X['seno']) + (X['interacao1'] * X['interacao2'] * X['interacao3'])

Assim como a análise exploratória e os testes de hipótese, vamos aplicar a seleção de features sempre nos dados de treino.

In [7]:
X_train , X_test, y_train , y_test = train_test_split(X, y , test_size=0.85 , random_state=0)

In [8]:
forest = RandomForestRegressor(n_jobs= -1 , max_depth = 5)

# fit boruta
boruta_selector = BorutaPy(forest , n_estimators = 50 , max_iter=10 , random_state=0)
boruta_selector.fit(np.array(X_train) , np.array(y_train))

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=50,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x2CBD83DB640),
         max_iter=10, n_estimators=50,
         random_state=RandomState(MT19937) at 0x2CBD83DB640)

In [9]:
# Terceiro filtro com as features selecionadas pelo boruta
cols_drop_boruta= [not x for x in boruta_selector.support_.tolist()] # apenas invertendo o vetor de true/false
cols_drop_boruta= X_train.loc[:,cols_drop_boruta].columns.tolist()

In [10]:
cols_drop_boruta

['seno',
 'interacao3',
 'aleatoria_1',
 'aleatoria_2',
 'aleatoria_3',
 'aleatoria_4']

## Recursive Feature Elimination

In [11]:
# 1 a Seleção Random Forest
rfecv_RFC = RFECV(estimator= forest , scoring = 'neg_mean_squared_error')
rfecv_RFC.fit(X_train,y_train)

RFECV(estimator=RandomForestRegressor(max_depth=5, n_estimators=50, n_jobs=-1,
                                      random_state=RandomState(MT19937) at 0x2CBD83DB640),
      scoring='neg_mean_squared_error')

In [12]:
print(rfecv_RFC.n_features_) # Número de features selecionadas como as mais importantes 
print(rfecv_RFC.support_) # Vetor de booleans. True se a variável deve ser mantida e False caso contrário
mask_RFC = rfecv_RFC.support_

2
[ True  True False False False False False False False False]


In [14]:
cols_drop_RFE= [not x for x in mask_RFC] # apenas invertendo o vetor de true/false
cols_drop_RFE= X_train.loc[:,cols_drop_RFE].columns.tolist()

In [15]:
cols_drop_RFE

['seno',
 'interacao1',
 'interacao2',
 'interacao3',
 'aleatoria_1',
 'aleatoria_2',
 'aleatoria_3',
 'aleatoria_4']

# Filter

O dataset credit se refere a clientes e seus empréstimos. A coluna target indica se o cliente honrou com seus compromissos ou não.

AS colunas com Sufixo PAY e BILL e a coluna LIMIT_BAL são numéricas. Assim, podemos testar as médias dessas features em relação aos grupos da variável target.

## ANOVA

## Chi2

# Prática Independente

Aplique o RFECV e o Boruta no dataset filtrado.